In [18]:
!pip install streamlit pyngrok scikit-learn matplotlib


In [19]:
!ngrok config add-authtoken 2zUv87gQUDDua9D97j5T8Nt0sd9_3ptdpiB8Xr2dHJDBaEtAv


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [20]:
import pickle
import os
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X, y = iris.data, iris.target

model = RandomForestClassifier()
model.fit(X, y)

os.makedirs("model", exist_ok=True)
with open("model/trained_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Model trained and saved to model/trained_model.pkl")


✅ Model trained and saved to model/trained_model.pkl


In [25]:
%%writefile app.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.datasets import load_iris

@st.cache_resource
def load_model():
    with open("model/trained_model.pkl", "rb") as f:
        return pickle.load(f)

def predict(model, data):
    return model.predict([data])[0], model.predict_proba([data])[0]

# Load model and dataset
model = load_model()
iris = load_iris()
feature_names = iris.feature_names

st.title("🌼 Iris Flower Classifier")
st.write("Enter flower measurements below to get a prediction.")

# 🧾 Form for user input
with st.form("input_form"):
    st.subheader("📥 Input Flower Measurements")

    sepal_length = st.number_input("Sepal Length (cm)", min_value=4.0, max_value=8.0, value=5.1)
    sepal_width = st.number_input("Sepal Width (cm)", min_value=2.0, max_value=4.5, value=3.5)
    petal_length = st.number_input("Petal Length (cm)", min_value=1.0, max_value=7.0, value=1.4)
    petal_width = st.number_input("Petal Width (cm)", min_value=0.1, max_value=2.5, value=0.2)

    submitted = st.form_submit_button("🔍 Predict")

if submitted:
    input_data = [sepal_length, sepal_width, petal_length, petal_width]

    # Predict
    label_idx, probabilities = predict(model, input_data)
    label_name = iris.target_names[label_idx]

    # Results
    st.subheader("✅ Prediction Result")
    st.success(f"The predicted species is: **{label_name.capitalize()}**")

    st.subheader("📊 Prediction Probabilities")
    fig, ax = plt.subplots()
    ax.bar(iris.target_names, probabilities, color=["skyblue", "lightgreen", "salmon"])
    ax.set_ylabel("Probability")
    ax.set_ylim(0, 1)
    st.pyplot(fig)

    st.subheader("📋 Your Input Summary")
    st.table({
        "Feature": feature_names,
        "Value": input_data
    })


Overwriting app.py


In [27]:
from pyngrok import ngrok
ngrok.kill()

!streamlit run app.py &>/content/logs.txt &

url = ngrok.connect(addr="8501", proto="http")
print("🚀 Your Streamlit app is live at:", url)


🚀 Your Streamlit app is live at: NgrokTunnel: "https://abe6-34-90-229-172.ngrok-free.app" -> "http://localhost:8501"
